In [3]:
import pandas as pd
import numpy as np
import os
import sys
import joblib
import re
import gc
from pathlib import Path
import yaml
import warnings
warnings.filterwarnings('ignore')
sys.path.append(os.path.abspath('..'))
from scripts.download_data import download_dataset
from sentinel.evaluation import SentinelEvaluator
from sentinel.inference import SentinelInference
from sentinel.modeling import SentinelTrainer
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
base_path = Path('../')

In [3]:
# # uncomment this cell and run it to download the dataset
# download_dataset(output_dir= base_path / 'data/raw', force=True)

#### **Train models**

In [19]:
df = pd.read_csv(base_path / 'data/raw/train_raw.csv')

In [20]:
gc.collect()
df.head(15).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
TransactionID,2987000,2987001,2987002,2987003,2987004,2987005,2987006,2987007,2987008,2987009,2987010,2987011,2987012,2987013,2987014
isFraud,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TransactionDT,86400,86401,86469,86499,86506,86510,86522,86529,86535,86536,86549,86555,86564,86585,86596
TransactionAmt,68.5,29.0,59.0,50.0,50.0,49.0,159.0,422.5,15.0,117.0,75.887,16.495,50.0,40.0,10.5
ProductCD,W,W,W,W,H,W,W,W,H,W,C,C,W,W,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
id_36,NaN,NaN,NaN,NaN,F,NaN,NaN,NaN,F,NaN,F,F,NaN,NaN,NaN
id_37,NaN,NaN,NaN,NaN,T,NaN,NaN,NaN,F,NaN,T,T,NaN,NaN,NaN
id_38,NaN,NaN,NaN,NaN,T,NaN,NaN,NaN,T,NaN,T,T,NaN,NaN,NaN
DeviceType,NaN,NaN,NaN,NaN,mobile,NaN,NaN,NaN,mobile,NaN,desktop,desktop,NaN,NaN,NaN


In [21]:
trainer = SentinelTrainer(base_path=base_path)

In [22]:
init_features = df.columns.tolist()
init_obj_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
cat_cols = [

    'ProductCD', 'card4', 'card6', 'DeviceType',
    'os_type', 'browser_type', 'device_vendor',
    'addr1', 'card1', 'card2',
    'P_emaildomain_vendor_id', 
    'P_emaildomain_suffix_id',
    
    'device_info_combo', 'card_email_combo', 
    'product_network_combo', 'card1_addr1_combo', 
    'os_browser_combo'
] + init_obj_cols

cat_cols = list(set(cat_cols))

In [23]:
data_split_config = {'train_size': 0.85,
 'nan_thresh': 0.95,
 'corr_thresh': 0.97, 
 'cat_cols': cat_cols}
trainer.prepare_data(df, **data_split_config)


🚀 Starting Data Preparation (Train Split=0.85)
   Train size: 501,959 (85.0%)
   Test size:  88,581 (15.0%)
 
  ⚙️  Running SentinelPreprocessing...
--- Fitting Sentinel Preprocessor ---
Dropped 9 columns
--- Sentinel Preprocessor Fitted ---
--- Transforming 501959 rows ---
--- Transforming 88581 rows ---
   🛠️  Running SentinelFeatureEngineering...
--- Fitting Sentinel v7.0 on 501,959 rows ---
   > PCA Group_0: 11 cols -> 2 comps (97% var)
   > PCA Group_1: 23 cols -> 6 comps (97% var)
   > PCA Group_2: 18 cols -> 5 comps (97% var)
   > PCA Group_3: 22 cols -> 7 comps (97% var)
   > PCA Group_4: 20 cols -> 7 comps (97% var)
   > PCA Group_5: 43 cols -> 10 comps (97% var)
   > PCA Group_6: 29 cols -> 7 comps (97% var)
   > PCA Group_7: 31 cols -> 8 comps (97% var)
   > PCA Group_8: 19 cols -> 7 comps (97% var)
   > PCA Group_9: 46 cols -> 8 comps (97% var)
   > PCA Group_10: 16 cols -> 6 comps (97% var)
   > PCA Group_11: 32 cols -> 10 comps (97% var)
   > PCA Group_12: 11 cols -> 8 c

In [24]:
X_train_eng, y_train, X_test_eng, y_test = trainer.X_train, trainer.y_train, trainer.X_test, trainer.y_test

In [25]:
X_train_eng.head(15).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
TransactionID,2987000,2987001,2987002,2987003,2987004,2987005,2987006,2987007,2987008,2987009,2987010,2987011,2987012,2987013,2987014
TransactionDT,86400,86401,86469,86499,86506,86510,86522,86529,86535,86536,86549,86555,86564,86585,86596
TransactionAmt,68.5,29.0,59.0,50.0,50.0,49.0,159.0,422.5,15.0,117.0,75.887001,16.495001,50.0,40.0,10.5
ProductCD,1,1,1,1,2,1,1,1,2,1,3,3,1,1,1
card1,13926,2755,4663,18132,4497,5937,12308,12695,2803,17399,16496,4461,3786,12866,11839
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PCA_Group_13_2,0.109226,0.109226,0.109226,0.109226,-0.816481,0.109226,0.109226,0.109226,-0.816481,0.109226,0.109226,0.109226,0.109226,0.109226,0.109226
PCA_Group_13_3,0.107902,0.107902,0.107902,0.107902,-0.839058,0.107902,0.107902,0.107902,-0.839058,0.107902,0.107902,0.107902,0.107902,0.107902,0.107902
PCA_Group_13_4,-0.093709,-0.093709,-0.093709,-0.093709,0.951159,-0.093709,-0.093709,-0.093709,0.951159,-0.093709,-0.093709,-0.093709,-0.093709,-0.093709,-0.093709
PCA_Group_13_5,0.022433,0.022433,0.022433,0.022433,-0.099798,0.022433,0.022433,0.022433,-0.099798,0.022433,0.022433,0.022433,0.022433,0.022433,0.022433


In [26]:
all_cols = X_train_eng.columns.tolist()
print('All columns: ', all_cols)
gc.collect()

All columns:  ['TransactionID', 'TransactionDT', 'TransactionAmt', 'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'dist1', 'dist2', 'P_emaildomain', 'R_emaildomain', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06', 'id_09', 'id_10', 'id_11', 'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_28', 'id_29', 'id_32', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'country', 'hour_of_day', 'day_of_week', 'day_of_month', 'month_year', 'P_emaildomain_length', 'P_emaildomain_has_digits', 'P_emaildomain_is_free', 'P_emaildomain_vendor_id', 'P_emaildomain_suffix_id', 'P_emaildomain_country_id', 'P_emaildomain_risk_score', 'R_emaildomain_length', 'R_emaildomain_has_digits',

0

In [27]:
D_cols = [f'D{i}' for i in range(1, 16) if f'D{i}' in X_train_eng.columns]
statistical_drops = [
    'C2', 'C3', 'C10', 'C11', 'C12',          
    'UID_amt_sum_1h', 'UID_count_24h',      
    'id_04', 'id_06', 'id_10', 'id_17',   
    'id_19', 'id_20',
    'TransactionAmt_log_z',                 
    'screen_height',                    
    'is_99_cents',                      
    'D9_norm',                     
    'addr1_switch',              
    'Amt_div_card4_mean'    
]

features_to_drop_final = [
    'TransactionID',   
    'TransactionDT',   
    'UID',          
    'UID_hash',        
    'P_emaildomain',   
    'R_emaildomain',   
    'country',
     
    'card1', 'card2', 'card3', 'card5', 
    'addr1', 'addr2',                  
    
    *D_cols,        

    'month_year',       
    'day_of_month',     
    'hour_of_day',      
    'days_since_first_txn', 
    'UID_dt_diff',     
    
    *statistical_drops,
    
    'multi_entity_sharing',
    'device_vendor_degree',
    'unique_addrs_per_card',
    'unique_emails_per_card',
    'cards_per_device',
    'addr1_degree',
    
    'card1_addr1_combo', 
    'device_info_combo', 
    'D10_norm',          
    'D15_norm',        
    
    'cents_value',                
    'UID_velocity_1h',            
    'UID_velocity_12h',
    'UID_count_1h',
    'UID_count_12h',
    'email_country_mismatch',     
    'P_emaildomain_country_id',   
    'R_emaildomain_country_id',
    'P_emaildomain_has_digits',
    'R_emaildomain_has_digits',
    
    'PCA_Group_0_1'
]

In [32]:
FIXED_PARAMS = {
            'lgb': {'objective': 'binary', 'boosting_type': 'gbdt', 'metric': 'auc', 'n_jobs': -1, 'verbose': -1, 'random_state': 42},
            'cb': {'loss_function': 'Logloss', 'eval_metric': 'AUC', 'grow_policy': 'SymmetricTree','bootstrap_type': 'Bernoulli', 'thread_count': -1, 'task_type': 'GPU', 'verbose': 0, 'random_state': 42},
            'xgb': {'objective': 'binary:logistic', 'booster': 'gbtree', 'eval_metric': 'auc', 'grow_policy': 'depthwise', 'device': 'cuda', 'tree_method': 'hist', 'early_stopping_rounds': 50, 'n_jobs': -1, 'verbosity': 0, 'random_state': 42}
        }

##### **Train LightGBM**

In [14]:
print(f"\n--- 1. Baseline Training for Feature Selection ---")
lgb_baseline_result = trainer.train_model(
    model_type='lgb',
    params=None,        
    top_n_features=None, 
    full_data=False,    
    n_folds=6,
    cols_to_drop=features_to_drop_final,
    save_artifacts=False
)


--- 1. Baseline Training for Feature Selection ---

🚀 Training LGB Model...

🚀 Training Final LGB on 211 features | Full Data: False

🚀 Training LGB Model...
   -> Fold 1/6...
   --- Size: Train=71711, Val=71708 ---
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Number of positive: 1922, number of negative: 69789
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.029696 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 39169
[LightGBM] [Info] Number of data points in the train set: 71711, number of used features: 211
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.026802 -> initscore=-3.592110
[LightGBM] [Info] Start training from score -3.592110
   -> Fold 2/6...
   --- Size: Train=143419, Val=71708 --

In [15]:
lgb_top_features = lgb_baseline_result['importances'].groupby('feature')['importance'].mean().sort_values(ascending=False).index.tolist()
lgb_best_params = trainer.optimize_hyperparameters(model_type='lgb', n_trials=50, n_folds=5, top_n_features=lgb_top_features[:150])


🔍 Optuna: Optimizing LGB on 150 features, (50 trials)...


[I 2026-01-22 03:20:14,453] A new study created in memory with name: no-name-e7dae8f9-56b8-44c4-bd97-b77d7064f74c
[I 2026-01-22 03:22:51,119] Trial 0 finished with value: 0.9300875574963087 and parameters: {'n_estimators': 4645, 'learning_rate': 0.006611715444395548, 'num_leaves': 79, 'max_depth': 15, 'subsample': 0.9274619020516957, 'colsample_bytree': 0.9086383602166552, 'reg_alpha': 0.027084401001824746, 'reg_lambda': 2.361433020361864, 'min_child_samples': 91}. Best is trial 0 with value: 0.9300875574963087.
[I 2026-01-22 03:23:25,610] Trial 1 finished with value: 0.9310766148083797 and parameters: {'n_estimators': 4709, 'learning_rate': 0.11189633756408782, 'num_leaves': 27, 'max_depth': 9, 'subsample': 0.5128542040433113, 'colsample_bytree': 0.7494197544472448, 'reg_alpha': 6.588856366839097, 'reg_lambda': 8.468989806207742e-07, 'min_child_samples': 98}. Best is trial 1 with value: 0.9310766148083797.
[I 2026-01-22 03:24:00,815] Trial 2 finished with value: 0.9346257937031869 and

   ✅ Best params: {'n_estimators': 3380, 'learning_rate': 0.018342265266958778, 'num_leaves': 201, 'max_depth': 11, 'subsample': 0.9755577454774549, 'colsample_bytree': 0.5028262928198358, 'reg_alpha': 0.00020441325409426117, 'reg_lambda': 0.0011288410876290117, 'min_child_samples': 86}


In [16]:
lgb_best_params.update(FIXED_PARAMS['lgb'])
lgb_report = trainer.final_train('lgb', lgb_best_params, lgb_top_features[:150])


🚀 Training LGB Model...

🏁 Final Test Results (LGB):
   Optimum Threshold: 0.0496
   AUC: 0.8711
   Precision: 56.45%
   Recall: 52.38%
   Net Profit Impact: $179,478.89
💾 Saved lgb model & 150 features.


##### **Train CatBoost**

In [17]:
print(f"\n--- 1. Baseline Training for Feature Selection ---")
cb_baseline_result = trainer.train_model(
    model_type='cb',
    params=None,        
    top_n_features=None, 
    full_data=False,    
    n_folds=6,
    cols_to_drop=features_to_drop_final,
    save_artifacts=False
)


--- 1. Baseline Training for Feature Selection ---

🚀 Training CB Model...

🚀 Training Final CB on 211 features | Full Data: False

🚀 Training CB Model...
   -> Fold 1/6...
   --- Size: Train=71711, Val=71708 ---


Default metric period is 5 because AUC is/are not implemented for GPU


   -> Fold 2/6...
   --- Size: Train=143419, Val=71708 ---


Default metric period is 5 because AUC is/are not implemented for GPU


   -> Fold 3/6...
   --- Size: Train=215127, Val=71708 ---


...

   -> Fold 5/6...
   --- Size: Train=358543, Val=71708 ---


Default metric period is 5 because AUC is/are not implemented for GPU


   -> Fold 6/6...
   --- Size: Train=430251, Val=71708 ---


Default metric period is 5 because AUC is/are not implemented for GPU



🏁 Final Test Set Evaluation:

🏁 Final Test Results (CB):
   Optimum Threshold: 0.3763
   AUC: 0.8620
   Precision: 44.46%
   Recall: 48.82%
   Net Profit Impact: $154,438.81


In [18]:
cb_top_features = cb_baseline_result['importances'].groupby('feature')['importance'].mean().sort_values(ascending=False).index.tolist()
cb_best_params = trainer.optimize_hyperparameters(model_type='cb', n_trials=20, n_folds=5, top_n_features=cb_top_features[:150])
cb_best_params.update(FIXED_PARAMS['cb'])


🔍 Optuna: Optimizing CB on 150 features, (20 trials)...


[I 2026-01-22 04:30:39,484] A new study created in memory with name: no-name-97b0a97f-e89d-4ee4-9938-ef7a49771c6e
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
[I 2026-01-22 04:33:21,360] Trial 0 finished with value: 0.9504572785576583 and parameters: {'iterations': 2154, 'learning_rate': 0.0956992919268524, 'depth': 8, 'l2_leaf_reg': 23.30958702110133, 'subsample': 0.9134901988629602, 'random_strength': 1.1328001999920832e-09}. Best is trial 0 with value: 0.9504572785576583.
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Defau

   ✅ Best params: {'iterations': 1636, 'learning_rate': 0.01822338863985892, 'depth': 10, 'l2_leaf_reg': 13.593530090603206, 'subsample': 0.7233747505340273, 'random_strength': 0.028636790883207173}


In [ ]:
cb_report = trainer.final_train('cb', cb_best_params, cb_top_features[:150])


🚀 Training CB Model...


Default metric period is 5 because AUC is/are not implemented for GPU



🏁 Final Test Results (CB):
   Optimum Threshold: 0.4159
   AUC: 0.8707
   Precision: 48.53%
   Recall: 50.96%
   Net Profit Impact: $172,559.87
💾 Saved cb model & 150 features.


##### **Train Xgboost**

In [29]:
print(f"\n--- 1. Baseline Training for Feature Selection ---")
xgb_baseline_result = trainer.train_model(
    model_type='xgb',
    params=None,        
    top_n_features=None, 
    full_data=False,    
    n_folds=6,
    cols_to_drop=features_to_drop_final,
    save_artifacts=False
)


--- 1. Baseline Training for Feature Selection ---

🚀 Training XGB Model...

🚀 Training Final XGB on 211 features | Full Data: False

🚀 Training XGB Model...
   -> Fold 1/6...
   --- Size: Train=71711, Val=71708 ---
   -> Fold 2/6...
   --- Size: Train=143419, Val=71708 ---
   -> Fold 3/6...
   --- Size: Train=215127, Val=71708 ---
   -> Fold 4/6...
   --- Size: Train=286835, Val=71708 ---
   -> Fold 5/6...
   --- Size: Train=358543, Val=71708 ---
   -> Fold 6/6...
   --- Size: Train=430251, Val=71708 ---

🏁 Final Test Set Evaluation:

🏁 Final Test Results (XGB):
   Optimum Threshold: 0.1189
   AUC: 0.8614
   Precision: 47.47%
   Recall: 47.45%
   Net Profit Impact: $155,840.48


In [ ]:
xgb_top_features = xgb_baseline_result['importances'].groupby('feature')['importance'].mean().sort_values(ascending=False).index.tolist()
xgb_best_params = trainer.optimize_hyperparameters(model_type='xgb', n_trials=20, n_folds=3, top_n_features=xgb_top_features[:150])



🔍 Optuna: Optimizing XGB on 150 features, (20 trials)...


[I 2026-01-22 10:36:10,606] A new study created in memory with name: no-name-a92c3853-3d8c-4b8b-bef6-700a730e38b7
[I 2026-01-22 10:36:44,288] Trial 0 finished with value: 0.9455878484362908 and parameters: {'n_estimators': 2953, 'learning_rate': 0.07664967673662174, 'max_depth': 7, 'min_child_weight': 8, 'subsample': 0.9326527387410064, 'colsample_bytree': 0.5257057246233434, 'gamma': 2.6227180488364527e-08, 'reg_alpha': 3.019058293721559e-07, 'reg_lambda': 1.727400424607608e-07}. Best is trial 0 with value: 0.9455878484362908.
[I 2026-01-22 10:39:37,458] Trial 1 finished with value: 0.9453064075634989 and parameters: {'n_estimators': 3502, 'learning_rate': 0.010555572732814083, 'max_depth': 5, 'min_child_weight': 7, 'subsample': 0.5668061500540388, 'colsample_bytree': 0.7057827086539722, 'gamma': 1.0794776964532876e-07, 'reg_alpha': 0.14766216893161951, 'reg_lambda': 2.3328301726128475e-05}. Best is trial 0 with value: 0.9455878484362908.
[I 2026-01-22 10:43:29,918] Trial 2 finished w

   ✅ Best params: {'n_estimators': 3118, 'learning_rate': 0.01797269754326933, 'max_depth': 9, 'min_child_weight': 5, 'subsample': 0.6600960698001002, 'colsample_bytree': 0.7207843217753258, 'gamma': 3.278589843556369e-06, 'reg_alpha': 8.367274561617439, 'reg_lambda': 0.031979841717789684}


In [39]:
xgb_best_params.update(FIXED_PARAMS['xgb'])
del xgb_best_params['early_stopping_rounds']
xgb_report = trainer.final_train('xgb', xgb_best_params, xgb_top_features[:150]) 


🚀 Training XGB Model...

🏁 Final Test Results (XGB):
   Optimum Threshold: 0.1585
   AUC: 0.9064
   Precision: 49.49%
   Recall: 59.29%
   Net Profit Impact: $225,897.42
💾 Saved xgb model & 150 features.


In [40]:
trainer.select_best_model()


🏆 Selecting Best Configuration based on Financial Impact...

   Evaluating 13 combinations using Cost Strategy...
   ⭐ New Leader: [lgb:1.0] Net Savings=$179,478.89
   ⭐ New Leader: [xgb:1.0] Net Savings=$225,897.42

🎉 WINNER: {'xgb': 1.0}
   Net Savings: $225,897.42
   Optimal Threshold: 0.1585
💾 Saved production_config.json to ../models/prod_v1


{'weights': {'xgb': 1.0},
 'threshold': 0.15848484848484848,
 'profit': 225897.42,
 'auc': 0.9063812313390824}

In [41]:
xgb_report

{'performance': {'precision': 0.4949,
  'recall': 0.5929,
  'fpr_insult_rate': 0.0218,
  'auc': 0.9064},
 'financials': {'fraud_stopped_val': 242618.44,
  'fraud_missed_val': 226990.08,
  'false_positive_loss': 62421.02,
  'net_savings': 225897.42},
 'counts': {'total_eval': 88581,
  'tp_count': 1828,
  'fp_count': 1866,
  'fn_count': 1255},
 'xgb': XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=0.7207843217753258, device='cuda',
               early_stopping_rounds=None, enable_categorical=True,
               eval_metric='auc', feature_types=None, feature_weights=None,
               gamma=3.278589843556369e-06, grow_policy='depthwise',
               importance_type=None, interaction_constraints=None,
               learning_rate=0.01797269754326933, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=9

#### **Post-processing & inference**

In [6]:
test_raw = pd.read_csv(base_path / 'data/raw/test_raw.csv')

In [47]:
inference  = SentinelInference(model_dir=base_path/'models/prod_v1')

2026-01-22 12:17:56,329 - INFO - Loading Sentinel artifacts from ../models/prod_v1...


2026-01-22 12:17:59,229 - INFO - Loaded xgb model.
2026-01-22 12:17:59,230 - INFO - ✅ Sentinel Inference initialized in 2.90s 


In [52]:
pred = inference.predict(test_raw)

--- Transforming 88581 rows ---
--- Transforming 88,581 rows ---


In [61]:
y_pred_prob = pred['probabilities']
y_true = test_raw['isFraud']
amounts = test_raw['TransactionAmt']

In [62]:
evaluator = SentinelEvaluator(y_true, y_pred_prob, amounts)

In [122]:
th = evaluator.find_best_threshold('friction', max_fpr=0.02)
th

0.1722

In [156]:
report = evaluator.report_business_impact(0.12)

In [157]:
report

{'performance': {'precision': 0.4342,
  'recall': 0.6257,
  'fpr_insult_rate': 0.0294,
  'auc': 0.9063},
 'financials': {'fraud_stopped_val': 265098.47,
  'fraud_missed_val': 204510.05,
  'false_positive_loss': 84170.19,
  'net_savings': 229153.28},
 'counts': {'total_eval': 88581,
  'tp_count': 1929,
  'fp_count': 2514,
  'fn_count': 1154}}

In [ ]:
["card_email_combo_fraud_rate", "PCA_Group_9_0", "C4", "C14", "C8", "card_email_combo", 
"PCA_Group_11_0", "C1", "C5", "PCA_Group_6_2", "M4", "PCA_Group_3_2", "R_emaildomain_is_free", "PCA_Group_6_1", 
"M5", "C7", "PCA_Group_13_0", "card1_freq_enc", "composite_risk_score", "PCA_Group_13_2", "PCA_Group_12_0", "C13", "C6", 
"PCA_Group_8_1", "PCA_Group_5_0", "PCA_Group_7_4", "PCA_Group_4_2", "device_info_combo_fraud_rate", "PCA_Group_1_2", 
"PCA_Group_13_4", "is_exact_dollars", "M6", "PCA_Group_8_5", "D2_norm", "PCA_Group_8_0", "PCA_Group_5_8",
 "os_browser_combo", "PCA_Group_6_6", "PCA_Group_5_4", "email_match_status", "id_12", "Amt_div_ProductCD_mean",
  "D8_norm", "PCA_Group_13_3", "PCA_Group_8_6", "R_emaildomain_length", "PCA_Group_2_2", "D1_norm", 
  "PCA_Group_12_4", "M2", "PCA_Group_9_4", "product_network_combo", "R_emaildomain_risk_score", "M3", 
  "ProductCD", "PCA_Group_4_1", "PCA_Group_13_1", "PCA_Group_3_0", "PCA_Group_12_7", "PCA_Group_2_0", 
  "addr1_fraud_rate", "PCA_Group_11_2", "PCA_Group_12_5", "PCA_Group_9_3", "PCA_Group_12_2", "id_09", 
  "PCA_Group_11_7", "PCA_Group_12_1", "PCA_Group_9_1", "PCA_Group_1_1", "PCA_Group_5_1", "UID_freq_enc", 
  "PCA_Group_11_4", "PCA_Group_10_4", "screen_area", "D4_norm", "C9", "PCA_Group_2_4", "Amt_div_addr1_mean",
   "PCA_Group_4_5", "P_emaildomain_suffix_id", "time_gap_anomaly", "PCA_Group_4_0", "PCA_Group_4_6", 
   "PCA_Group_3_1", "PCA_Group_3_3", "PCA_Group_1_3", "P_emaildomain_length", "PCA_Group_4_3", "PCA_Group_12_3", 
   "screen_aspect_ratio", "PCA_Group_5_7", "D6_norm", "R_emaildomain_suffix_id", "PCA_Group_11_5", 
   "PCA_Group_12_6", "PCA_Group_3_4", "PCA_Group_8_4", "PCA_Group_4_4", "TransactionAmt_decimal", "dist1", 
   "PCA_Group_3_6", "device_vendor", "PCA_Group_2_3", "Amt_div_P_emaildomain_vendor_id_mean", "id_18", 
   "PCA_Group_2_1", "os_type", "PCA_Group_10_3", "PCA_Group_5_2", "PCA_Group_10_0", "P_emaildomain_vendor_id", 
   "R_emaildomain_vendor_id", "PCA_Group_1_5", "PCA_Group_11_8", "PCA_Group_1_4", "txn_sequence", "PCA_Group_13_5", 
   "PCA_Group_3_5", "Amt_div_card1_mean", "id_36", "id_03", "D11_norm", "PCA_Group_11_3", "D12_norm", "D13_norm", 
   "PCA_Group_8_3", "D3_norm", "PCA_Group_9_6", "id_05", "card6", "addr1_freq_enc", "PCA_Group_5_6", "PCA_Group_1_0", 
   "D5_norm", "TransactionAmt_log", "PCA_Group_11_6", "id_01", "M9", "id_02", "PCA_Group_10_2", "PCA_Group_10_1", 
   "hour_of_day_fraud_rate", "PCA_Group_5_3", 
"PCA_Group_11_1", "TransactionAmt", "screen_width", "PCA_Group_5_9", "D7_norm", "PCA_Group_7_0"]